In [1]:
from pathlib import Path
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pyquaternion import Quaternion
import cv2
import pandas as pd

import time
from collections import deque, defaultdict

import habitat
from habitat.config import Config
from habitat.config.default import get_config
from habitat.core.env import Env
from habitat.sims.habitat_simulator.actions import HabitatSimActions

# from habitat_baselines.rl.ddppo.policy.resnet import ResNet, resnet50, se_resneXt50
# from habitat_baselines.rl.ppo import PointNavBaselinePolicy
from habitat_baselines.agents.ppo_agents import PPOAgent
from gym.spaces import Box, Dict, Discrete

root = Path('../frames')

In [2]:
models = {
#     'rgb':   '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-2plus-mp3d-train-val-test-se-resneXt50-rgb.pth',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-2plus-resnet50.pth',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-4plus-resnet50.pth',
    'rgb':   '/Users/nimit/Documents/robomaster/habitat/models/v2/rgb.pth',
    'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/depth.pth'
}

configs = {
    'rgb':   '/Users/nimit/Documents/robomaster/habitat/habitat2robomaster/rgb_test.yaml',
    'depth': '/Users/nimit/Documents/robomaster/habitat/habitat2robomaster/depth_test.yaml',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml'
}

In [3]:
c = Config()

c.RESOLUTION       = 256
c.HIDDEN_SIZE      = 512
c.RANDOM_SEED      = 7

c.INPUT_TYPE       = 'depth'
c.MODEL_PATH       = models[c.INPUT_TYPE]
c.GOAL_SENSOR_UUID = 'pointgoal_with_gps_compass'

c.freeze()

env = Env(config=get_config(configs[c.INPUT_TYPE]))
agent = PPOAgent(c)

2020-04-04 05:00:42,593 Initializing dataset PointNav-v1
2020-04-04 05:00:42,771 initializing sim Sim-v0
I0404 05:00:44.705297 91444 simulator.py:142] Loaded navmesh /Users/nimit/Documents/robomaster/habitat/habitat-api/data/scene_datasets/habitat-test-scenes/skokloster-castle.navmesh
2020-04-04 05:00:44,713 Initializing task Nav-v0


In [11]:
agg_metrics: Dict = defaultdict(float)

num_episodes = 100
count_episodes = 0
while count_episodes < num_episodes:
    agent.reset()
    observations = env.reset()

    i = 1
    prev_pos = env.sim.get_agent_state().position
    prev_rot = env.sim.get_agent_state().rotation
    
    d_pos = deque(maxlen=10)
    d_rot = deque(maxlen=10)
    d_col = deque(maxlen=5)
    while not env.episode_over:
        action = agent.act(observations)
#         print(observations['rgb'], action) # dataset
#         if action['action'] == 0:
#             print('TERMINATE')
#             time.sleep(1)
        observations = env.step(action)
        cv2.imshow('rgb', observations['rgb'])
        cv2.waitKey(1)
        print(torch.cat([torch.Tensor([*env.current_episode.start_position, *env.current_episode.start_rotation, *env.current_episode.goals[0].position])]))
        
        d_pos.append(np.linalg.norm(env.sim.get_agent_state().position - prev_pos))
        d_rot.append((env.sim.get_agent_state().rotation - prev_rot).absolute())
        d_col.append(int(env.get_metrics()['collisions']['is_collision']))
        if (i > 10 and np.max(d_pos) == 0 and np.max(d_rot) == 0):
            print('STUCK')
            break
        if (i > 5 and np.min(d_col) == 1):# 0.2):
            print('COLLIDE')
            break

        time.sleep(.2)
        i += 1
        prev_pos = env.sim.get_agent_state().position
        prev_rot = env.sim.get_agent_state().rotation

    metrics = env.get_metrics()
#     print(metrics)
    for m, v in metrics.items():
        if m != 'collisions':
            agg_metrics[m] += v
    count_episodes += 1

cv2.destroyWindow('rgb')

avg_metrics = {k: v / count_episodes for k, v in agg_metrics.items()}
avg_metrics

tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+01, 0.0000e+00, 1.2696e-02, 0.0000e+00,
        9.9992e-01, 5.5421e+00, 9.9186e-03, 1.9877e+01])
tensor([5.6774e+00, 2.0992e-01, 1.2240e+0

KeyboardInterrupt: 

## DDPPO

In [1]:
from pathlib import Path
import torch

from habitat.config.default import get_config
from habitat import get_config as get_task_config

from habitat_baselines.rl.ddppo.algo.ddppo_trainer import DDPPOTrainer
from habitat_baselines.rl.ddppo.policy.resnet_policy import PointNavResNetPolicy
from habitat_baselines.common.env_utils import construct_envs
from habitat_baselines.common.environments import get_env_class

In [2]:
torch.load('/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-2plus-resnet50.pth')['model_args']

Namespace(backbone='resnet50', hidden_size=512, num_recurrent_layers=2, resnet_baseplanes=32, rnn_type='LSTM', sensors='DEPTH_SENSOR')

In [3]:
ddppo_config = get_config('habitat_baselines/config/pointnav/ddppo_pointnav.yaml')
ddppo_config.defrost()
ddppo_config.TASK_CONFIG = get_task_config(ddppo_config.BASE_TASK_CONFIG_PATH)
ddppo_config.freeze()

ppo_cfg = ddppo_config.RL.PPO
trainer = DDPPOTrainer(ddppo_config)

2020-04-02 04:11:16,203 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav_gibson.yaml
CHECKPOINT_FOLDER: data/new_checkpoints
CHECKPOINT_INTERVAL: 50
DATASET:
  CONTENT_SCENES: ['*']
  DATA_PATH: data/datasets/pointnav/habitat-test-scenes/v1/{split}/{split}.json.gz
  SCENES_DIR: data/scene_datasets
  SPLIT: train
  TYPE: PointNav-v1
ENVIRONMENT:
  ITERATOR_OPTIONS:
    CYCLE: True
    GROUP_BY_SCENE: True
    MAX_SCENE_REPEAT_EPISODES: -1
    MAX_SCENE_REPEAT_STEPS: 10000
    NUM_EPISODE_SAMPLE: -1
    SHUFFLE: True
    STEP_REPETITION_RANGE: 0.2
  MAX_EPISODE_SECONDS: 10000000
  MAX_EPISODE_STEPS: 1000
ENV_NAME: NavRLEnv
EVAL_CKPT_PATH_DIR: data/new_checkpoints
LOG_FILE: test.log
LOG_INTERVAL: 10
NUM_PROCESSES: 4
NUM_UPDATES: 10000
PYROBOT:
  BASE_CONTROLLER: proportional
  BASE_PLANNER: none
  BUMP_SENSOR:
    TYPE: PyRobotBumpSensor
  DEPTH_SENSOR:
    CENTER_CROP: False
    HEIGHT: 480
    MAX_DEPTH: 5.0
    MIN_DEPTH: 0.0
    NORMALIZE_DEPTH: True
    TYPE: PyRobotDepthSensor


In [4]:
trainer.train()
# trainer._setup_actor_critic_agent(ppo_cfg)
# envs = construct_envs(ddppo_config, get_env_class('NavRLEnv'))

2020-04-02 04:11:16,404 Initializing dataset PointNav-v1


EOFError: 

In [5]:
self.actor_critic = PointNavResNetPolicy(
    observation_space=envs.observation_spaces[0],
    action_space=envs.action_spaces[0],
    hidden_size=ppo_cfg.hidden_size,
    rnn_type=ddppo_config.RL.DDPPO.rnn_type,
    num_recurrent_layers=ddppo_config.RL.DDPPO.num_recurrent_layers,
    backbone=ddppo_config.RL.DDPPO.backbone,
    goal_sensor_uuid=ddppo_config.TASK_CONFIG.TASK.GOAL_SENSOR_UUID,
    normalize_visual_inputs="rgb"
    in envs.observation_spaces[0].spaces,
)

NameError: name 'envs' is not defined